In [47]:
import requests
import time
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

In [40]:
url = "https://www.cupoy.com/MixNewsMongoAction.do?op=getTopMixNewsByBucketGroup&groupid=business_tw&startNewsID=1&len=3"  #財經新聞
resp = requests.get(url)
resp.encoding = 'utf-8'
# print(resp.text)  # 如有需要，列印出網頁原始碼

In [48]:
j = json.loads(resp.text)
length = len(j['items'])
for i in tqdm(range(length)):
    print(j['items'][i]['title'])

100%|██████████| 3/3 [00:00<00:00, 6058.21it/s]

辭去七年工作，最後一敗塗地的創業夢，卻成為我的人生轉捩點 ｜ 詹宇／讀者投書 ｜ 換日線 Crossing
 - Meet 創業小聚
【打死我也發不出口罩】3000 萬日供應量 vs. 28 億的日需求！中國口罩廠員工「日做 16 小時，壓力大到哭」 | BuzzOrange
